In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

ruta = "../../data/processed_data/casen_subset.csv"

df = pd.read_csv(ruta)

In [2]:
df.head()

,estrato,region,r1a,r1a_esp,r1a_esp_cod,s13,s13_fonasa
0,1630324,16,1,NaN,NaN,1,2.0
1,1630324,16,1,NaN,NaN,1,2.0
2,1630324,16,1,NaN,NaN,1,3.0
3,1630324,16,1,NaN,NaN,1,1.0
4,1630324,16,1,NaN,NaN,1,-88.0


In [3]:
df["nivel_economico"] = df["estrato"].astype(str).str[-1].astype(int)
df["zona"] = df["estrato"].astype(str).str[-2].astype(int)
df["comuna"] = df["estrato"].astype(str).str[:-2].astype(int)
df.head()

,estrato,region,r1a,r1a_esp,r1a_esp_cod,s13,s13_fonasa,nivel_economico,zona,comuna
0,1630324,16,1,NaN,NaN,1,2.0,4,2,16303
1,1630324,16,1,NaN,NaN,1,2.0,4,2,16303
2,1630324,16,1,NaN,NaN,1,3.0,4,2,16303
3,1630324,16,1,NaN,NaN,1,1.0,4,2,16303
4,1630324,16,1,NaN,NaN,1,-88.0,4,2,16303


In [4]:
# Conteo de niveles económicos por comuna
niveles = (
    df.groupby("comuna")["nivel_economico"]
    .value_counts()
    .unstack(fill_value=0)
)

# Conteo de zonas por comuna
zonas = (
    df.groupby("comuna")["zona"]
    .value_counts()
    .unstack(fill_value=0)
)

niveles, zonas

(nivel_economico     1     2     3    4    5    6     7
 comuna                                                
 1101              505  2964  1066  106    0    0     0
 1107             1877     0     0    0    0    0  1216
 1401                0     0     0  260  191    0     0
 1402                0     0     0   87    0    0     0
 1403               74     0     0    0    0    0     0
 ...               ...   ...   ...  ...  ...  ...   ...
 16301             199     0     0  330    0    0   336
 16302               0     0     0  152    0  294     0
 16303              33     0     0  196    0    0     0
 16304             109     0     0   37    0    0     0
 16305               0     0     0  153    0    0     0
 
 [335 rows x 7 columns],
 zona       1    2
 comuna           
 1101    4535  106
 1107    3053   40
 1401     260  191
 1402       0   87
 1403       0   74
 ...      ...  ...
 16301    535  330
 16302    152  294
 16303     33  196
 16304     37  109
 16305     68   8

In [5]:
import numpy as np
# Calcula el total por comuna
valor = [1, 2, 3, 4, 5, 6, 7]

niveles["total_personas"] = niveles[valor].sum(axis=1)

# Promedio ponderado del nivel socioeconómico
niveles["nivel_promedio"] = (
    (niveles[valor] * np.array(valor)).sum(axis=1) / niveles["total_personas"]
)
niveles.head()

nivel_economico,1,2,3,4,5,6,7,total_personas,nivel_promedio
comuna,,,,,,,,,
1101,505,2964,1066,106,0,0,0,4641,2.166559
1107,1877,0,0,0,0,0,1216,3093,3.358875
1401,0,0,0,260,191,0,0,451,4.423503
1402,0,0,0,87,0,0,0,87,4.000000
1403,74,0,0,0,0,0,0,74,1.000000


In [6]:
# Asegurarte que sea un array
valor = np.array([1, 2, 3, 4, 5, 6, 7])

# Asegurar que el promedio esté en forma (n, 1)
mean = niveles["nivel_promedio"].to_numpy().reshape(-1, 1)

# Expandir valor a (1, 7) para que haga broadcast con (n, 1)
valor = valor.reshape(1, -1)

# Calcular varianza ponderada
var = (
    (niveles[[1,2,3,4,5,6,7]].to_numpy() * (valor - mean) ** 2).sum(axis=1)
    / niveles["total_personas"]
)

# Desviación estándar como desigualdad
niveles["desviacion_estandar"] = np.sqrt(var)

In [7]:
niveles.head()

nivel_economico,1,2,3,4,5,6,7,total_personas,nivel_promedio,desviacion_estandar
comuna,,,,,,,,,,
1101,505,2964,1066,106,0,0,0,4641,2.166559,0.634131
1107,1877,0,0,0,0,0,1216,3093,3.358875,2.930693
1401,0,0,0,260,191,0,0,451,4.423503,0.494114
1402,0,0,0,87,0,0,0,87,4.000000,0.000000
1403,74,0,0,0,0,0,0,74,1.000000,0.000000


In [8]:
valor_zona = [1, 2]

zonas["total_zona"] = zonas[valor_zona].sum(axis=1)
# Calcula el total por comuna

# indice de ruralidad
zonas["ruralidad"] = (
    (zonas[valor_zona] * np.array(valor_zona)).sum(axis=1) / zonas["total_zona"] -1
)
zonas.head()

zona,1,2,total_zona,ruralidad
comuna,,,,
1101,4535,106,4641,0.022840
1107,3053,40,3093,0.012932
1401,260,191,451,0.423503
1402,0,87,87,1.000000
1403,0,74,74,1.000000


In [9]:
ruta_comunas = "../../data/processed_data/codigo_comuna.csv"
df_comuna = pd.read_csv(ruta_comunas)
df_comuna.head()

,Código comuna,Comuna
0,15101,Arica
1,15102,Camarones
2,15201,Putre
3,15202,General Lagos
4,1101,Iquique


In [10]:
# Asegúrate de que los índices sean columnas
zonas = zonas.reset_index()
niveles = niveles.reset_index()

# Unir zona y niveles por código de comuna
df_merge = zonas.merge(niveles, on="comuna", how="inner")

# Unir con el dataframe de códigos para obtener el nombre de la comuna
df_final = df_merge.merge(df_comuna, left_on="comuna", right_on="Código comuna", how="left")

# Dejar solo las columnas necesarias
df_final = df_final[["Código comuna","Comuna", "ruralidad", "nivel_promedio", "desviacion_estandar"]]

# Ver resultado
print(df_final.head())

   Código comuna         Comuna  ruralidad  nivel_promedio  \
0           1101        Iquique   0.022840        2.166559   
1           1107  Alto Hospicio   0.012932        3.358875   
2           1401   Pozo Almonte   0.423503        4.423503   
3           1402         Camiña   1.000000        4.000000   
4           1403       Colchane   1.000000        1.000000   

   desviacion_estandar  
0             0.634131  
1             2.930693  
2             0.494114  
3             0.000000  
4             0.000000  


In [14]:
df_final.to_csv("../../data/processed_data/casen_niveles.csv", index=False)

In [11]:
df.head()

,estrato,region,r1a,r1a_esp,r1a_esp_cod,s13,s13_fonasa,nivel_economico,zona,comuna
0,1630324,16,1,NaN,NaN,1,2.0,4,2,16303
1,1630324,16,1,NaN,NaN,1,2.0,4,2,16303
2,1630324,16,1,NaN,NaN,1,3.0,4,2,16303
3,1630324,16,1,NaN,NaN,1,1.0,4,2,16303
4,1630324,16,1,NaN,NaN,1,-88.0,4,2,16303


In [12]:
# Conteo de niveles económicos por comuna
niveles = (
    df.groupby("region")["nivel_economico"]
    .value_counts()
    .unstack(fill_value=0)
)

# Conteo de zonas por comuna
zonas = (
    df.groupby("region")["zona"]
    .value_counts()
    .unstack(fill_value=0)
)

niveles, zonas

(nivel_economico     1      2     3     4    5     6     7
 region                                                   
 1                2603   2964  1066   651  191     0  1216
 2                2204   4400   964  1109    0   352     0
 3                3510   1899   689  1448    0  1118   347
 4                2357   2195   609  2018    0   386   671
 5                2578   6161  2075  6324    0  2544   870
 6                1725   2399   783  6487    0  1369  1101
 7                3249   2807   811  4915    0    70  1887
 8                5725   4970  2006  4583   70  1247  1313
 9                3572   1796   727  5349    0   772  1361
 10               2012   2405   807  3849    0  1392   242
 11                  0    194     0  1183    0  2337    33
 12                  0    226   826  2977    0   907    93
 13               4064  10306  6734  7247    0  4506  5817
 14               1710   2283   820  4470    0   907   413
 15               3022   3779   593   758    0     0    

In [13]:
# Calcula el total por comuna
valor = [1, 2, 3, 4, 5, 6, 7]

niveles["total_personas"] = niveles[valor].sum(axis=1)

# Promedio ponderado del nivel socioeconómico
niveles["nivel_promedio"] = (
    (niveles[valor] * np.array(valor)).sum(axis=1) / niveles["total_personas"]
)
niveles.head()

nivel_economico,1,2,3,4,5,6,7,total_personas,nivel_promedio
region,,,,,,,,,
1,2603,2964,1066,651,191,0,1216,8691,2.738465
2,2204,4400,964,1109,0,352,0,9029,2.264260
3,3510,1899,689,1448,0,1118,347,9011,2.697148
4,2357,2195,609,2018,0,386,671,8236,2.872632
5,2578,6161,2075,6324,0,2544,870,20552,3.297733


In [14]:
# Asegurarte que sea un array
valor = np.array([1, 2, 3, 4, 5, 6, 7])

# Asegurar que el promedio esté en forma (n, 1)
mean = niveles["nivel_promedio"].to_numpy().reshape(-1, 1)

# Expandir valor a (1, 7) para que haga broadcast con (n, 1)
valor = valor.reshape(1, -1)

# Calcular varianza ponderada
var = (
    (niveles[[1,2,3,4,5,6,7]].to_numpy() * (valor - mean) ** 2).sum(axis=1)
    / niveles["total_personas"]
)

# Desviación estándar como desigualdad
niveles["desviacion_estandar"] = np.sqrt(var)

In [15]:
valor_zona = [1, 2]

zonas["total_zona"] = zonas[valor_zona].sum(axis=1)
# Calcula el total por comuna

# indice de ruralidad
zonas["ruralidad"] = (
    (zonas[valor_zona] * np.array(valor_zona)).sum(axis=1) / zonas["total_zona"] -1
)
zonas.head()

zona,1,2,total_zona,ruralidad
region,,,,
1,8021,670,8691,0.077091
2,8211,818,9029,0.090597
3,7947,1064,9011,0.118078
4,6276,1960,8236,0.237980
5,16697,3855,20552,0.187573


In [17]:
# Asegúrate de que los índices sean columnas
zonas = zonas.reset_index()
niveles = niveles.reset_index()

# Unir zona y niveles por código de comuna
df_merge = zonas.merge(niveles, on="region", how="inner")



# Dejar solo las columnas necesarias
df_final = df_merge[["region", "ruralidad", "nivel_promedio", "desviacion_estandar"]]

# Ver resultado
df_final.head(10)

,region,ruralidad,nivel_promedio,desviacion_estandar
0,1,0.077091,2.738465,1.967768
1,2,0.090597,2.264260,1.181571
2,3,0.118078,2.697148,1.889598
3,4,0.237980,2.872632,1.834501
4,5,0.187573,3.297733,1.676887
5,6,0.307126,3.659911,1.691888
6,7,0.296164,3.245142,1.921268
7,8,0.204931,2.864216,1.801053
8,9,0.356706,3.307063,1.897360
9,10,0.311292,3.239470,1.676343


In [18]:
df_final.to_csv("../../data/processed_data/casen_niveles_regiones.csv", index=False)